### Presentation Types:  Fuga, ID, and PEN

* February 2022 Version.  Key Features:

    * Uses getDistance to identify `close matches` with side-by-side comparison of soggetti.  With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same. These are reported as "flexed entries" in a separate column.


    * Labels Fuga, PEn, and ID according to time intervals.  
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)

In [10]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
import itertools
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### The following are special functions used by the classifier.  Don't change them.

#### Load the Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [16]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# just add the CRIM Piece ID here
mei_file = 'CRIM_Model_0015.mei'


url = git_prefix + mei_file
# piece = importScore('Music_Files/Senfl_Ave_forCRIM.mei_msg.mei')
piece = importScore(url)
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')

print(piece.metadata)


Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0015.mei
{'title': 'Quo abiit dilectus tuus', 'composer': 'Manchicourt, Pierre de'}


#### Run the Classifier Here

In [17]:
include_hidden_types = False
combine_unisons = True
melodic_ngram_length = 4
edit_distance_threshold = 1
nr = piece.getNoteRest(combineUnisons=combine_unisons)
dur = piece.getDuration(df=nr)
mel = piece.getMelodic(df=nr, kind='d', end=False)
dur_ng = piece.getNgrams(df=dur, n=melodic_ngram_length)
mel_ng = piece.getNgrams(df=mel, n=melodic_ngram_length)
entries = piece.getEntries(mel_ng)
output = classify_entries_as_presentation_types(piece, nr, dur_ng, entries, edit_distance_threshold, include_hidden_types)


In [18]:
output

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,"Manchicourt, Pierre de",Quo abiit dilectus tuus,0.0,"[1/1.0, 3/1.0, 10/1.0, 12/2.0]","[P-4, P-5, P-4]","[0.0, 16.0, 72.0, 90.0]","[5, -2, 2, 2, 4, -2, 2, 2, 4, -3, 2, 2, 3, -2,...","[16.0, 56.0, 18.0]","[Superior, Contratenor, Tenor, Bassus]",FUGA,4,True
1,"Manchicourt, Pierre de",Quo abiit dilectus tuus,38.0,"[5/4.0, 7/2.0, 7/4.0, 10/4.0, 14/4.0, 16/2.0, ...","[P-5, P-4, P8, P-8, P-5, M9, P4, P-8]","[38.0, 50.0, 54.0, 78.0, 110.0, 122.0, 126.0, ...","[-3, 2, -2, -2]","[12.0, 4.0, 24.0, 32.0, 12.0, 4.0, 12.0, 8.0]","[Superior, Contratenor, Bassus, Superior, Teno...",FUGA,9,False
2,"Manchicourt, Pierre de",Quo abiit dilectus tuus,174.0,"[22/4.0, 24/2.0, 26/4.0, 28/2.0, 29/4.0]","[P5, P-12, P5, P5]","[174.0, 186.0, 206.0, 218.0, 230.0]","[3, -2, -2, -2, 2, -2, -2, -2]","[12.0, 20.0, 12.0, 12.0]","[Contratenor, Superior, Bassus, Tenor, Contrat...",FUGA,5,True
3,"Manchicourt, Pierre de",Quo abiit dilectus tuus,266.0,"[34/2.0, 38/2.0, 40/2.0]","[P1, P-5]","[266.0, 298.0, 314.0]","[2, -3, 5, -5]","[32.0, 16.0]","[Bassus, Tenor, Bassus]",FUGA,3,False
4,"Manchicourt, Pierre de",Quo abiit dilectus tuus,274.0,"[35/2.0, 36/2.0, 41/2.0]","[P-5, P1]","[274.0, 282.0, 322.0]","[2, -3, 5, -2]","[8.0, 40.0]","[Superior, Contratenor, Tenor]",FUGA,3,False
5,"Manchicourt, Pierre de",Quo abiit dilectus tuus,342.0,"[43/4.0, 48/1.0]",[P-8],"[342.0, 376.0]","[2, -2, 4, -2]",[34.0],"[Superior, Tenor]",FUGA,2,False
6,"Manchicourt, Pierre de",Quo abiit dilectus tuus,366.0,"[46/4.0, 51/2.0, 53/2.0, 54/2.0, 54/4.0]","[P-8, P12, P-8, P4]","[366.0, 402.0, 418.0, 426.0, 430.0]","[-2, -2, -2, -2]","[36.0, 16.0, 8.0, 4.0]","[Contratenor, Bassus, Superior, Bassus, Tenor]",FUGA,5,False
7,"Manchicourt, Pierre de",Quo abiit dilectus tuus,446.0,"[56/4.0, 58/2.0, 60/4.0, 61/4.0, 63/2.0, 64/2....","[P5, P-8, P-5, P5, P8, P-8]","[446.0, 458.0, 478.0, 486.0, 498.0, 506.0, 518.0]","[4, 2, 2, -3]","[12.0, 20.0, 8.0, 12.0, 8.0, 12.0]","[Contratenor, Superior, Tenor, Bassus, Contrat...",FUGA,7,False
8,"Manchicourt, Pierre de",Quo abiit dilectus tuus,534.0,"[67/4.0, 68/2.0, 70/2.0, 72/2.0, 74/2.0, 74/4....","[P4, M2, P-5, P1, P4, P8, P-8, m-7, m7, P-4, P...","[534.0, 538.0, 554.0, 570.0, 586.0, 590.0, 604...","[-2, -2, -2, 2, -2, -2, -3, 2]","[4.0, 16.0, 16.0, 16.0, 4.0, 14.0, 10.0, 16.0,...","[Bassus, Superior, Contratenor, Tenor, Bassus,...",FUGA,16,True
9,"Manchicourt, Pierre de",Quo abiit dilectus tuus,680.0,"[85/1.0, 87/1.0, 90/1.0, 92/1.0]","[P-5, P-4, P-5]","[680.0, 696.0, 720.0, 736.0]","[-3, 2, 2, 2, -2, 2, 2, 2]","[16.0, 24.0, 16.0]","[Superior, Contratenor, Tenor, Bassus]",ID,4,True


#### Run Classifier on Several Pieces at Once
Results are combined into a single dataframe

In [5]:


git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')
piece_list =  ['CRIM_Mass_0005_3.mei',
             'CRIM_Mass_0005_4.mei',
             'CRIM_Mass_0005_5.mei',
             'CRIM_Model_0001.mei',
             'CRIM_Mass_0002_1.mei',
             'CRIM_Mass_0002_2.mei',
             'CRIM_Mass_0002_3.mei',
             'CRIM_Mass_0002_4.mei',
             'CRIM_Mass_0002_5.mei',
             'CRIM_Model_0015.mei',
             'CRIM_Mass_0013_1.mei',
             'CRIM_Mass_0013_2.mei',
             'CRIM_Mass_0013_3.mei',
             'CRIM_Mass_0013_4.mei',
             'CRIM_Mass_0013_5.mei',
             'CRIM_Model_0019.mei',
             'CRIM_Mass_0019_1.mei',
             'CRIM_Mass_0019_2.mei',
             'CRIM_Mass_0019_3.mei',
             'CRIM_Mass_0019_4.mei',
             'CRIM_Mass_0019_5.mei']


In [6]:
include_hidden_types = False
melodic_ngram_length = 4
edit_distance_threshold = 1
final = pd.DataFrame()
for work in piece_list:
    url = git_prefix + work
    piece = importScore(url)   
    nr = piece.getNoteRest()
    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=melodic_ngram_length)
    mel_ng = piece.getMelodicEntries(interval_settings=('d', True, True), n=melodic_ngram_length)
    output = classify_entries_as_presentation_types(piece, nr, dur_ng, mel_ng, edit_distance_threshold, include_hidden_types)
    final = final.append(output, ignore_index=True)
final.head()

mei.base: WARNING: Importing <slur> without @startid and @endid is not yet supported.


Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_3.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_4.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0005_5.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0001.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0002_1.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0002_2.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0002_3.mei
Successfully imported https://raw.githubusercontent.com/CRIM-Project/C

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,Antoine de Févin,Missa Ave Maria: Credo,654.0,"[82/4.0, 83/2.0, 86/2.0, 86/4.0, 89/4.0, 92/2.0]","[M-6, m-3, M-6, M13, P-5]","[654.0, 658.0, 682.0, 686.0, 710.0, 730.0]","[-2, -3, 2, 2, -2, -2, 2, 2]","[4.0, 24.0, 4.0, 24.0, 20.0]","[Sup[erius], Altus, Tenor, Bassus, Sup[erius],...",FUGA,6,True
1,Antoine de Févin,Missa Ave Maria: Credo,190.0,"[24/4.0, 25/2.0]",[P5],"[190.0, 194.0]","[-2, -3, 3, -2]",[4.0],"[Bassus, Tenor]",FUGA,2,False
2,Antoine de Févin,Missa Ave Maria: Credo,876.0,"[109/3.0, 111/3.0, 112/3.0]","[P-8, P-5]","[876.0, 892.0, 900.0]","[2, -2, -2, -2, 1, -2, -2, -2]","[16.0, 8.0]","[Sup[erius], Tenor, Bassus]",FUGA,3,True
3,Antoine de Févin,Missa Ave Maria: Credo,1144.0,"[142/1.0, 144/1.0]",[P-5],"[1144.0, 1160.0]","[1, -2, -2, 2, 1, -2, -3, 2]",[16.0],"[Sup[erius], Altus]",FUGA,2,True
4,Antoine de Févin,Missa Ave Maria: Credo,462.0,"[58/4.0, 59/2.0]",[M-6],"[462.0, 466.0]","[1, 1, 1, -2]",[4.0],"[Sup[erius], Altus]",FUGA,2,False


#### Save to your folder of CSV's here in the Jupyter Hub
#### You can save as CSV, or as Excel.
#### You will then need to download this to your computer to view it properly

* Note that in the following part of the code below, you will need to give your file a name:

`saved_csv/**file_name**.xlsx`

In [26]:
writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')
points.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [6]:
points.to_csv('saved_csv/your_file_title.csv')